In [ ]:
#| default_exp payment_clients


# payment clients

> Payment clients are the interfaces and implementations that allow users to pay for L402 offers.

In [ ]:
#| export

from abc import ABC, abstractmethod
from datetime import datetime
from typing import List, Dict, Any

from fastcore.utils import *
import fastcore.basics as fc
import os
import json
from l402.payment_providers import *
from l402.utils import *
import httpx
from pydantic import BaseModel


In [ ]:
chain = 'base-mainnet'

In [ ]:
#| export

class PaymentProvider(ABC):
    """Base class for all payment providers"""
    supported_methods: list[str] = []  # Will be overridden by each provider
    
    @abstractmethod
    def pay(self, *args, **kwargs):
        pass

class PaymentRequest(BaseModel):
    """Represents a payment request to get payment details from a payment provider"""
    offer_id: str
    payment_context_token: str
    payment_method: str
    chain: str = ""
    asset: str = ""


In [ ]:
# from l402.utils import *

# run_l402_server(PaymentRequest, port=9000)

## Coinbase Provider

In [ ]:
#| export

from cdp import *

class CoinbaseProvider(fc.BasicRepr):
    def __init__(self, wallet: Wallet,
                asset: str = 'usdc'):
        store_attr()
        self.supported_methods=["onchain"]
        self.chain = self.wallet.network_id

    # NOTE: in the fewsats example, we will return an async taskID
    def pay(self,
            amount: float,
            address: str,
            asset: str):
        # TODO, return a generic async task 
        return self.wallet.transfer(amount, asset, address).wait()

In [ ]:
c = CoinbaseProvider(wallet=create_test_wallet(fund=False, chain=chain))
c

CoinbaseProvider(wallet=Wallet: (id: ef60974b-2571-44ca-ac25-ff99d2e3c88f, network_id: base-mainnet, server_signer_status: None), asset='usdc', supported_methods=['onchain'], chain='base-mainnet')

In [ ]:
r = httpx.get('https://l402-offers.replit.app')
# r = httpx.get('http://localhost:9000/offers')
r.status_code, r.text


(402,
 '{"offers":[{"offer_id":"ce23eefd-1156-4aa4-85c1-361918a24485","amount":1,"currency":"USD","description":"Purchase 1 credit for API access","title":"1 Credit Package","type":"one-time","payment_methods":["onchain","lightning"]}],"payment_context_token":"8ee0082b-97c8-4ac6-baa4-f9f544dc977c","payment_request_url":"https://hub-5n97k.ondigitalocean.app/v0/l402/payment-request","version":"0.2.2"}')

In [ ]:
o = r.json()
o

{'offers': [{'offer_id': 'ce23eefd-1156-4aa4-85c1-361918a24485',
   'amount': 1,
   'currency': 'USD',
   'description': 'Purchase 1 credit for API access',
   'title': '1 Credit Package',
   'type': 'one-time',
   'payment_methods': ['onchain', 'lightning']}],
 'payment_context_token': '8ee0082b-97c8-4ac6-baa4-f9f544dc977c',
 'payment_request_url': 'https://hub-5n97k.ondigitalocean.app/v0/l402/payment-request',
 'version': '0.2.2'}

In [ ]:
data = {
    "offer_id": first(o['offers'])['offer_id'],
    "payment_method": 'onchain',
    "chain": chain,
    "asset": 'usdc',
    "payment_context_token": o['payment_context_token']
    }
r = httpx.post(o['payment_request_url'], json=data)
r.status_code, r.json()


(200,
 {'expires_at': '2025-01-28T02:06:17.637705+00:00',
  'offer_id': 'ce23eefd-1156-4aa4-85c1-361918a24485',
  'payment_request': {'checkout_url': 'https://commerce.coinbase.com/pay/be427ef1-153a-495e-93eb-1b10550731d0',
   'address': '0x03059433BCdB6144624cC2443159D9445C32b7a8',
   'chain': 'base-mainnet',
   'asset': 'usdc'},
  'version': '0.2.2'})

In [ ]:
data = {
    "offer_id": first(o['offers'])['offer_id'],
    "payment_method": 'lightning',
    "payment_context_token": o['payment_context_token']
    }
r = httpx.post(o['payment_request_url'], json=data, timeout=15)
r.status_code, r.text


(200,
 '{"expires_at": "2025-01-28T04:14:16.328280+00:00", "offer_id": "ce23eefd-1156-4aa4-85c1-361918a24485", "payment_request": {"lightning_invoice": "lnbc90n1pnes5txpp5zxhg46zr2lycmqg9km930h9mmefeyanw3jm8c2cxpqxlhlvevkgsdq6xysyxun9v35hggzsv93kkct8v5cqzpgxqrzpjrzjqwghf7zxvfkxq5a6sr65g0gdkv768p83mhsnt0msszapamzx2qvuxqqqqz99gpz55yqqqqqqqqqqqqqq9qrzjq25carzepgd4vqsyn44jrk85ezrpju92xyrk9apw4cdjh6yrwt5jgqqqqz99gpz55yqqqqqqqqqqqqqq9qsp5y9unj8j6czfjy274dj7war7k7xey8jvgegsxcjenct8ntah4eyvs9qxpqysgqq60qg26s74k4jsetcz954dlpghg63a8qrprqytr0ml0f2hxtfm0zujk4t54x2n4hxaxaz6d75tm7xjalv287mn559uq8tnmyfs92gdqpdutj9l"}, "version": "0.2.2"}')

In [ ]:
data = {
    "offer_id": 'test-lightning-1',
    "payment_method": 'lightning',
    "payment_context_token": '550cdc77-bdae-410b-bd7a-091b14be72bb'
    }
r = httpx.post('http://localhost:8000/v0/l402/payment-request', json=data, timeout=15)
r.status_code, r.text


(200,
 '{"expires_at": "2025-01-28T02:46:35.422815+00:00", "offer_id": "test-lightning-1", "payment_request": {"lightning_invoice": "lnbc90n1pnes0xhpp5lsp46vm2j47f8z3rm3lmj2lh54gvc4jn3hql0qhp9ryxef9fz0lsdpy23jhxapqf35kw6r5de5kueeq2pshjmt9de6qcqzpgxqrzpnrzjqwghf7zxvfkxq5a6sr65g0gdkv768p83mhsnt0msszapamzx2qvuxqqqqz99gpz55yqqqqqqqqqqqqqq9qrzjq25carzepgd4vqsyn44jrk85ezrpju92xyrk9apw4cdjh6yrwt5jgqqqqz99gpz55yqqqqqqqqqqqqqq9qsp5ykrsqs0x9mjt22937a0hn5wz0z0fxd9yxv97sud2kkrswuakh2js9qxpqysgqeq8g08w8n3t0wlhj3h3zkjsnpa8gznzncf98nqk92lyd96a30gsy8fxzk5q9z47fsy59ljn548xlck84qeta88wygppq3zed8qsz9dgqxn7rtq"}, "version": "0.2.2"}')

In [ ]:
#| export

def get_payment_request(payment_request_url: str,
                        payment_context_token: str,
                        offer_id: str, 
                        payment_method: str, 
                        chain: str = "", 
                        asset: str = ""):
    data = {
        "offer_id": offer_id,
        "payment_method": payment_method,
        "chain": chain,
        "asset": asset,
        "payment_context_token": payment_context_token
    }
    r = httpx.post(payment_request_url, json=data)
    r.raise_for_status()
    return r.json()


In [ ]:
r = get_payment_request(o['payment_request_url'], o['payment_context_token'], o['offers'][0]['offer_id'], 'onchain', chain, 'usdc')
r

ReadTimeout: The read operation timed out

In [ ]:
r = get_payment_request(o['payment_request_url'], o['payment_context_token'], o['offers'][0]['offer_id'], 'lightning')
r


HTTPStatusError: Server error '500 Internal Server Error' for url 'https://hub-5n97k.ondigitalocean.app/v0/l402/payment-request'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

In [ ]:
#| export

class Client(fc.BasicRepr):
    def __init__(self, lightning_provider = None, 
                 credit_card_provider = None, 
                 onchain_provider = None,
                 fewsats_provider = None):
        store_attr()
        self.lightning_provider = lightning_provider
        self.credit_card_provider = credit_card_provider
        self.onchain_provider = onchain_provider
        self.fewsats_provider = fewsats_provider


    def pay(self, ofr_body: dict): # ofr is the l402 offers response dictionary
        "Pay for an offer"
        # this actually does 3 things
        # 1. Selects offer
        # 2. Gets payment request details
        # 3. Uses user-provided payment method
        
        ofr = L402Response(**ofr_body)
        if len(ofr.offers) != 1: raise ValueError("Only one offer is supported")
        o = first(ofr.offers)

        if self.fewsats_provider:
            return self.fewsats_provider.pay(ofr_body)
        elif 'onchain' in o.payment_methods and self.onchain_provider:
            r = get_payment_request(ofr.payment_request_url, ofr.payment_context_token, o.offer_id, 'onchain', self.onchain_provider.chain, self.onchain_provider.asset)

            return self.onchain_provider.pay(o.amount, r['payment_request']['address'], r['payment_request']['asset'])
            
        # elif 'lightning' in o.payment_methods and self.lightning_provider:
        # elif 'credit_card' in o.payment_methods and self.credit_card_provider:
        else:
            raise ValueError(f"No payment provider available for {ofr.offers[0].payment_methods}")


In [ ]:
w = create_test_wallet(fund=False, chain=chain)
c = Client(onchain_provider=CoinbaseProvider(wallet=w, asset='usdc'))
try:
    c.pay(o)
except Exception as e:
    print(e)


Insufficient funds: have 0, need 1.


## Fewsats Client

In [ ]:
#| export

# class PaymentStatus:
#     PENDING = "pending"
#     COMPLETED = "completed"
#     FAILED = "failed"
#     EXPIRED = "expired"

#| export

class Fewsats:
    def __init__(self, api_key: str = None, base_url: str = "https://hub-5n97k.ondigitalocean.app"):
        self.api_key = api_key or os.environ.get("FEWSATS_API_KEY")
        if not self.api_key:
            raise ValueError("API key not provided and FEWSATS_API_KEY environment variable is not set")
        self.base_url = base_url
        self.client = httpx.Client()
        self.client.headers.update({"Authorization": f"Token {self.api_key}"})


    def _request(self, method: str, endpoint: str, **kwargs) -> Dict[str, Any]:
        url = f"{self.base_url}/{endpoint}"
        response = self.client.request(method, url, **kwargs)
        response.raise_for_status()
        return response.json()

    def get_payment_methods(self) -> List[Dict[str, Any]]:
        """Retrieve the user's payment methods.
        
        Returns:
            List[Dict[str, Any]]: A list of payment methods associated with the user's account.
        """
        return self._request("GET", "v0/stripe/payment-methods")


    def pay(self, ofr: dict):
        data = {
            "payment_request_url": ofr["payment_request_url"],
            "payment_context_token": ofr["payment_context_token"],
            "payment_method": "onchain",
            "offer": first(ofr["offers"])
        }
        print(data)
        print(self.base_url)
        print(self.api_key)
        return self._request("POST", "v0/l402/purchases/from-offer", json=data)

In [ ]:
f = Fewsats()
f = Fewsats(base_url='http://localhost:8000', api_key='yQbJPudW-nqiTyx880t8G83oLGyoR-RlnrLDmEe0D58')
f.get_payment_methods()

[{'id': 1,
  'last4': '4242',
  'brand': 'visa',
  'exp_month': 12,
  'exp_year': 2034,
  'is_default': False},
 {'id': 4,
  'last4': '4242',
  'brand': 'Visa',
  'exp_month': 12,
  'exp_year': 2034,
  'is_default': True}]

In [ ]:
c = Client(fewsats_provider=Fewsats(base_url='http://localhost:8000', api_key='yQbJPudW-nqiTyx880t8G83oLGyoR-RlnrLDmEe0D58'))
err = None
try:
    c.pay(o)
except Exception as e:
    print(e.response.text)

{'payment_request_url': 'https://hub-5n97k.ondigitalocean.app/v0/l402/payment-request', 'payment_context_token': 'a1700b1a-8325-47a9-902a-f90e82d21427', 'payment_method': 'onchain', 'offer': {'offer_id': 'f52bcccd-1057-4204-b1ae-01707bfe29af', 'amount': 1, 'currency': 'USD', 'description': 'Purchase 1 credit for API access', 'title': '1 Credit Package', 'type': 'one-time', 'payment_methods': ['onchain']}}
http://localhost:8000
yQbJPudW-nqiTyx880t8G83oLGyoR-RlnrLDmEe0D58
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/ninja/operation.py", line 341, in run
    result = await self.view_func(request, **values)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/hub_api/l402/api/common/decorators.py", line 17, in wrapper
    return await func(request, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/hub_api/l402/api/v0/views.py", line 32, in l402_create_purchase_from_offer_v0
    return await purchase_from_offer(


In [ ]:
err.response.json()

AttributeError: 'NoneType' object has no attribute 'response'

In [ ]:
try:
    f.pay(o)
except Exception as e:
    print(e)

name 'f' is not defined


In [ ]:
#| export

@patch
def _pay_onchain(self: Fewsats, address: str,
                    amount: str,
                    chain: str = '',
                    asset: str = ''):
    data = {
        "address": address,
        "amount": str(amount),
        "chain": chain,
        "asset": asset,
    }
    print(data)
    return self._request("POST", "v0/l402/purchases/onchain", json=data)


In [ ]:
f = Fewsats()
try:
    f._pay_onchain(address=r['payment_request']['address'], amount="0.000001", chain='base-mainnet', asset='usdc')
except Exception as e:
    print(e)

{'address': '0x114113186ca748Ea629A08B762df34a93f4C7e64', 'amount': '0.000001', 'chain': 'base-mainnet', 'asset': 'usdc'}
The read operation timed out


In [ ]:
o, r

({'offers': [{'amount': 1,
    'currency': 'USD',
    'description': 'Purchase 1 credit for API access',
    'offer_id': '33d08b4a-1589-4519-a24f-ba67e2166f0b',
    'payment_methods': ['onchain'],
    'title': '1 Credit Package',
    'type': 'one-time'}],
  'payment_context_token': 'd5607f07-3ecd-4f84-a9f9-74db7f922760',
  'payment_request_url': 'http://localhost:9000/payment_request',
  'version': '0.2.2'},
 {'expires_at': '2025-01-25T18:21:42.339161+00:00',
  'offer_id': '33d08b4a-1589-4519-a24f-ba67e2166f0b',
  'payment_request': {'address': '0xAa4b26Ca04692E6cAA310Dc05Feaf1dE75943d62',
   'chain': 'base-mainnet',
   'asset': 'usdc'},
  'version': '0.2.2'})